In [1]:
# Authors: Alexandre Gramfort
#          Chloe Clavel
# License: BSD Style.
# TP Cours ML Telecom ParisTech MDI343

import os.path as op
import numpy as np
import time

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cross_validation import KFold, cross_val_score

In [2]:
# Chargment des données

print("Loading dataset")

from glob import glob
filenames_neg = sorted(glob(op.join('data', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join('data', 'imdb1', 'pos', '*.txt')))
stopwords = open(op.join('data','english.stop')).read().split('\n')

texts_neg = [open(f).read() for f in filenames_neg]
texts_pos = [open(f).read() for f in filenames_pos]
texts = texts_neg + texts_pos
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0

print("%d documents" % len(texts))

Loading dataset
2000 documents


# Implémentation du classifieur

Definition de la fonction cleanText (question 5)

In [3]:
# La definition de la fonction cleanText a ete deplacee ici
# pour inclure l'option del_stopwords a la fonction count_words

# La fonction cleanText retire les stopwords contenu dans le fichier english.stop
# des textes contenus dans la liste rawTxt ainsi que la regex '\n'

def cleanText(rawTxt):
    cleanTxt = ' '.join(filter(lambda x: x.lower() not in stopwords,
                               rawTxt.split()))
    
    return cleanTxt.replace('\n', '')  # .replace(r'\W', '')

### Question 1

In [4]:
def count_words(texts, del_stopwords):
    """Vectorize text : return count of each word in the text snippets

    Parameters
    ----------
    texts : list of str
        The texts

    Returns
    -------
    vocabulary : dict
        A dictionary that points to an index in counts for each word.
    counts : ndarray, shape (n_samples, n_features)
        The counts of each word in each text.
        n_samples == number of documents.
        n_features == number of words in vocabulary.
    """
    
    if del_stopwords:
        # cette partie a ete ajoute pour la question 5
        cleanedTexts = list(map(cleanText, texts))
    else:
        cleanedTexts = texts
  
    words = set(' '.join(cleanedTexts).split())
    vocabulary = {k: i for i, k in enumerate(words)} 
    n_features = len(words)
    counts = np.zeros((len(cleanedTexts), n_features))

    for l, text in enumerate(cleanedTexts):
        for wd in text.split():
            counts[l, vocabulary[wd]] += 1

    return vocabulary, counts

### Question 2

Les classes positives et négatives ont été assignées à l'aide d'un classifieur qui tente de reconnaître les notes attribuées par les auteurs des commentaires.  
Etant donnée la diversité des systèmes de notations utilisés par les auteurs des commentaires, le label des commentaires (positif ou negatif) a été attribué à partir de la  de la manière suivante :

- Les notes (numérique ou sous forme d'étoiles) doivent préciser l'échelle de notation :
    Ex : "8/10", "four out of five", and "OUT OF ****: ***"

- Pour un système de notation à 5 étoiles (et les système numériques compatibles) :
    Les notes de 3,5 étoiles et plus sont considérées comme positives
    Les notes de 2 étoiles et moins sont considérées comme négatives

- Pour un système de notation à 4 étoiles (et les système numériques compatibles) :
    Les notes de 3 étoiles et plus sont considérées comme positives
    Les notes de 1,5 étoiles et moins sont considérées comme négatives

- Pour un système de notation avec des lettres:
	Les notes à partir de B sont considérées comme postives
	Les notes jusqu'à C- sont considérée comme négatives

De cette manière, l'algorythme n'est entrainé (et testé) qu'à partir de commentaire associés à des notes extrêmes (très basses ou très hautes). Les commentaires associés à des notes autour de la moyenne ont été exclus de la base.

### Question 3

In [5]:
class NB(BaseEstimator, ClassifierMixin):  
    def __init__(self):
        pass
  
    def fit(self, X, y):
        self.labels = np.array([(i, k) for i, k
                                in enumerate(set(y))], 
                          dtype=[('index', np.int),
                                 ('label', np.int)])
        # labels permets de distinguer les indices
        # (compris entre 0 et le nb de labels-1)
        # et les labels de y 
        # (qui pourrait être {-1, 1} ou encore {5, 9, 70})
        # il doivent seulement être des int
        
        n = X.shape[0]
        self.prior = np.empty(len(self.labels))
        self.condprob = np.empty((len(self.labels), X.shape[1]))

        for c in self.labels:
            nC = y[y==c['label']].shape[0]
            self.prior[c['index']] = nC / n
            T = np.sum(X[y==c['label']], axis=0)
            self.condprob[c['index']] = (T + 1) / np.sum(T + 1)
    
        return self

    def predict(self, X):
        score = np.empty((X.shape[0], len(self.labels)))
        score[:,:] = np.log(self.prior)

        for nzR, nzC in np.transpose(np.nonzero(X)):
            score[nzR] += np.log(self.condprob.T[nzC])

        # la fonction predict renvoie les labels tels qu'ils étaient
        # dans l'argument y de la fonction fit
        return self.labels[np.argmax(score, axis=1)]['label']
        
    def score(self, X, y):
        return np.mean(self.predict(X) == y)

In [6]:
# Comptage des mots dans les textes bruts
vocabulary, X = count_words(texts, False)

In [7]:
# Test sur la moitie des commentaires de l'algorithme Naive Bayes
# implemente et entraine sur l'autre moitie des donnees

nb = NB()
nb.fit(X[::2], y[::2])
print("Training score : {:.2%}".format(nb.score(X[::2], y[::2])))
print("Test score : {:.2%}".format(nb.score(X[1::2], y[1::2])))

Training score : 99.60%
Test score : 83.60%


### Question 4

In [8]:
%%timeit -n1 -r1

# Calcul des K-Folds sur le text brut :
# implémentation à partir de la fonction KFolds de sklearn (v17.)

kf = KFold(X.shape[0], n_folds=5, shuffle=True)
kFoldsRes = []
for train_indices, test_indices in kf:

    train_X = X[train_indices, :]
    train_y = y[train_indices]
    test_X = X[test_indices, :]
    test_y = y[test_indices]

    nbk = NB()
    nbk.fit(train_X, train_y)
    kFoldsRes.append(nbk.score(test_X, test_y))

print("Scores de la validation croisee du classifieur Naive Bayes implemente",
      "(a partir des textes bruts):",
      "\n".join('{}: {:.2%}'.format(*k) for k in enumerate(kFoldsRes)),
      "\nIC de l'accuracy : {0:.2%}% +/- {1:.4%}".format(
        np.mean(kFoldsRes),
        np.std(kFoldsRes)))

Scores de la validation croisee du classifieur Naive Bayes implemente (a partir des textes bruts): 0: 79.50%
1: 85.25%
2: 81.25%
3: 81.25%
4: 80.75% 
IC de l'accuracy : 81.60%% +/- 1.9339%
1 loop, best of 1: 14.2 s per loop


In [9]:
%%timeit -n1 -r1

# Calcul des K-Folds sur les textes bruts :
# à partir des fonctions KFolds et cross_val_score de sklearn (v17.)

kf = KFold(len(X), n_folds=5, shuffle=True)
cl_scores = cross_val_score(nb, X, y, cv=kf)
print("Scores de la validation croisee du classifieur Naive Bayes implemente",
      "(a partir des textes bruts):\n",
      "\n".join('{}: {:.2%}'.format(*k) for k in enumerate(cl_scores)),
      "\n IC de l'accuracy : {0:.2%} +/- {1:.4%}".format(
        np.mean(cl_scores),
        np.std(cl_scores)))

Scores de la validation croisee du classifieur Naive Bayes implemente (a partir des textes bruts):
 0: 81.50%
1: 82.50%
2: 83.25%
3: 81.25%
4: 81.75% 
 IC de l'accuracy : 82.05% +/- 0.7314%
1 loop, best of 1: 15.8 s per loop


Les deux dernieres cellules ont exactement le meme objectif :  
evaluer l'algorithme du Naive Bayes sur la problématique de sentiment analysis à partir des commentaires de films  
  
De plus, ils utilisent exactement la même méthode :  
une validation croisee sur 5 tirages.  
  
La premiere a ete implementee uniquement a partir de la fonction KFold de sklearn alors que la deuxieme utilise la fonction cross_val_score. On verifie qu'on obtient des résultats et des temps d'execution très similaires (qui varient faiblement du fait du shuffle aleatoire). Ce shuffle garantit que chacun des folds soit equilibre entre les avis positifs et les avis negatifs. Pour la suite nous utiliserons, pour des questions pratiques, la fonction cross_val_score de sklearn.    
  
On obtient ainsi une classification correcte à plus de 80% à partir des commentaires bruts.

### Question 5

In [10]:
%%timeit -n1 -r1

# Comptage des mots dans les documenst nettoyes
# a l'aide la fonction cleanText
cl_vocabulary, cl_X = count_words(texts, True)

# Cross-validation du classifieur Naive Bayes implemente
kf = KFold(len(X), n_folds=5, shuffle=True)
cl_scores = cross_val_score(nb, X, y, cv=kf)

print("Scores de la validation croisee du classifieur Naive Bayes implemente",
      "(a partir des textes sans stop words):\n",
      "\n".join('{}: {:.2%}'.format(*k) for k in enumerate(cl_scores)),
      "\n IC de l'accuracy : {0:.2%} +/- {1:.4%}".format(
        np.mean(cl_scores),
        np.std(cl_scores)))

Scores de la validation croisee du classifieur Naive Bayes implemente (a partir des textes sans stop words):
 0: 78.75%
1: 83.00%
2: 82.00%
3: 81.25%
4: 83.25% 
 IC de l'accuracy : 81.65% +/- 1.6171%
1 loop, best of 1: 38.8 s per loop


Les resultats ci-dessus sont obtenus a partir des commentaires pretraites avec la fonction implementee cleanText qui retirent les stop words (listes dans le fichier 'english.stop') des commentaires de films ainsi que la regex '\n'.  
  
On peut en effet, penser que ces mots courants sont utilises indifferemment dans des commentaires positifs et des des commentaires negatifs. Ils n'apportent donc logiquement aucune information sur l'avis de l'auteur du commentaire par rapport au film. En les retirant, on espere diminuer l'eventuel bruit qu'il generait et permettre a l'algoritme de se concentrer sur les autres mots.  
Je me suis permis de supprimer egalement la regex '\n' des commentaires afin que les mots qui y sont accoles puissent etre reunis avec les eventuelles autres occurences de ce mot. Ainsi, "\ndamn" est compte comme une occurence du mot "damn". L'ensemble des autres caracteres sont conserves.    
  
En conclusion, on constate que, sur 5 tirages aleatoires, l'amelioration du score de l'algorithme de Naive Bayes est très faible. Les stop words produisaient un bruit quasiment negligeable.

# Utilisation de scikit-learn

### Question 1

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [12]:
# La fonction display_CVResults sera utilisee pour les prochaines questions
# afin d'afficher les scores d'une validation croisee
# pour differents modeles sur les commentaires
# (qui ont subit des pretraitements differents)
def display_CVResults(model, texts, nb_folds):
    kf = KFold(len(X), n_folds=nb_folds, shuffle=True)
    cv_NBscores = cross_val_score(model, texts, y, cv=kf)
    
    print("Scores de la validation croisee :\n",
          "\n".join('{}: {:.2%}'.format(*k) for k in enumerate(cv_NBscores)),
          "\n IC de l'accuracy : {0:.2%} +/- {1:.4%}".format(
            np.mean(cv_NBscores),
            np.std(cv_NBscores)))

In [13]:
%%timeit -n1 -r1

# Cross-validation du classifieur Naive Bayes de sklearn
# a partir des textes pretraites avec la classe CountVectorizer()
# les stop words sont ceux de la classe CountVectorizer()
# les features sont les mots contenus dans les commentaires

NB_model = Pipeline([
        ('cntVect', CountVectorizer(stop_words={'english'})),
        ('nbCl', MultinomialNB())
    ])

display_CVResults(NB_model, texts, 5)

Scores de la validation croisee :
 0: 81.00%
1: 84.00%
2: 81.50%
3: 78.75%
4: 77.50% 
 IC de l'accuracy : 80.55% +/- 2.2605%
1 loop, best of 1: 12 s per loop


En utilisant les classes CountVectorizer et MultinomialNB de sklearn, les scores restent proches de 80%.  
Comme on peut le constater, l'algorithme implemente dans sklearn est plus rapide mais permet d'obtenir des resultats très similaires à ceux de la classe NB.
    
Les petites differences entre les scores sont majoritairement dues aux options de la classe CountVectorizer. Par defaut, la classe CountVectorizer ne tient pas compte de la ponctuation et les features sont les mots (au sens de la regex '\w'). Au contraire, la fonction implementee count_words considere un mot comme une chaine de caractere separee par un espace. La premiere definition pose pose differents problemes :
- elle considere les mots avec des tirets comme 2 mots differents alors qu'on peut considerer que la connotation (positive ou negative) decoule de leur association
- elle ne prend pas en compte la ponctuation qui peut apporter des informations sur l'avis de l'auteur du commentaire (ex : "???", "!!!!", ":S")  
  
Cependant, la seconde methode presente egalement des inconvenients puisqu'en tenant compte de la ponctuation, elle differencie par exemple "mot." de "mot".  
  
Empiriquement, la fonction implementee semble capables d'atteindre des scores légèrement superieurs et plus stable (par rapport a leur ecart-type). Pour d'avantage, de details, voir la partie "Analyse et test sur les methodes de vectorisation" à la fin de ce notebook.  
  
On remarque qu'on utilise desormais l'option "stop_words={'english'}" pour retirer les stop words. La difference avec l'option  "stop_words=stopwords" est cependant negligeable et n'a pas de réelle influence sur le score. 

In [14]:
%%timeit -n1 -r1

# Cross-validation du classifieur Naive Bayes de sklearn
# a partir des textes pretraites avec la classe CountVectorizer()
# les stop words sont ceux de la classe CountVectorizer()
# les features sont les caracteres contenus dans les commentaires

NB_model = Pipeline([
        ('cntVect', CountVectorizer(stop_words={'english'},
                                    ngram_range=(1, 2),
                                    token_pattern=r'\b\w+\b',
                                    min_df=1)),
        ('nbCl', MultinomialNB())
    ])

display_CVResults(NB_model, texts, 5)

Scores de la validation croisee :
 0: 84.75%
1: 82.25%
2: 84.25%
3: 83.25%
4: 82.00% 
 IC de l'accuracy : 83.30% +/- 1.0770%
1 loop, best of 1: 48.4 s per loop


La creation de la matrice des variables explicatives en utilisant les bigrammes permet de prendre en compte les associations de mots (par pair). En effet, avec le code précédent, les variables explicatives comprennent les mots individuellement mais aussi les pairs de mots qui se suivent dans une phrase. Le modèle tient donc compte des associations d'idées et permet d'isoler des expressions. Dans les faits, ce prétraitement a un impact positif sur les scores. On peut cependant deplorer la multiplication du nombre de variables explicatives qui rend la matrice encore plus sparse.

In [15]:
%%timeit -n1 -r1

# Cross-validation du classifieur Naive Bayes de sklearn
# a partir des textes pretraites avec la classe CountVectorizer()
# les stop words sont ceux de la classe CountVectorizer()
# les features sont les caracteres contenus dans les commentaires

NB_model = Pipeline([
        ('cntVect', CountVectorizer(stop_words={'english'},
                                    analyzer='char')),
        ('nbCl', MultinomialNB())
    ])

display_CVResults(NB_model, texts, 5)

Scores de la validation croisee :
 0: 62.50%
1: 59.75%
2: 61.50%
3: 62.25%
4: 60.00% 
 IC de l'accuracy : 61.20% +/- 1.1336%
1 loop, best of 1: 37.3 s per loop


Avec l'option "analyzer='char'", CountVectorizer construit une matrice ou les features sont des caracteres. L'algorithme du Naive Bayes estiment donc empiriqument les probabilites qu'un caractere se trouve dans un commentaire sachant que celui est positif ou negatif. Il estime ensuite la probabilite que les commentaires soient positifs ou negatifs en fonction des caracteres qu'ils contiennent pour la comparer avec les probabilites a priori.  
Or la relation entre l'avis d'une personne sur un film et l'ocurrence des caracteres dans son commentaire semble ne pas etre fondee. On observe en effet, que cette strategie ne permet de classifier les commentaires correctement que dans 60% des cas. De plus, son accuracy semble plus eleve que lorsqu'on utilise les mots. Le fait que ce classifieur soit un peu plus efficace que le hasard (qui correspondrait à un score de 50%) peut, à mon sens, s'expliquer par le fait que certains caracteres speciaux puissent être d'avantage associe a un type de commentaire.

### Question 2

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
# Cross-validation d'une Regression Logistique de sklearn
# a partir des textes pretraites avec la classe CountVectorizer()
# les stop words sont ceux de la classe CountVectorizer()

LR_model = Pipeline([
        ('cntVect', CountVectorizer(stop_words={'english'},
                                    ngram_range=(1, 2),
                                    token_pattern=r'\b\w+\b',
                                    min_df=1)),
        ('lrCl', LogisticRegression())
    ])

display_CVResults(LR_model, texts, 5)

Scores de la validation croisee :
 0: 85.50%
1: 86.25%
2: 82.50%
3: 86.50%
4: 83.50% 
 IC de l'accuracy : 84.85% +/- 1.5780%


La regression logistique (avec une penalisation quadratique et un intercep) permet d'atteindre un score légèrement superieur au classifieur du Naive Bayes. Ces deux algorirhmes sont adaptes pour une premiere approche du text mining.  

### Question 3

In [18]:
from nltk import SnowballStemmer

In [19]:
# pretraitement des textes :
# on remplace chaque mot par sa racine
# grace à la fonction stem de la classe SnowballStemmer

stemmer = SnowballStemmer("english")
st_texts = [" ".join([stemmer.stem(word) for word
                      in sentence.split()]) for sentence in texts]
# Apres de nombreux test la fonction map ne permet pas de réaliser
# cette tâche plus rapidement

# Cross-validation de la Regression Logistique de sklearn
# a partir des text pretraites obtenus
display_CVResults(LR_model, st_texts, 5)

Scores de la validation croisee :
 0: 83.25%
1: 86.50%
2: 85.25%
3: 86.25%
4: 83.50% 
 IC de l'accuracy : 84.95% +/- 1.3546%


La fonction stem de la classe SnowballStemmer permet de trouver la racine des mots dictionnaire (anglais dans notre cas). Cette fonction est utilisee ici pour regrouper les mots par rapport à leur racine afin de reduire le nombre de mots differents (et donc le nombre de features). On espere ainsi affiner les probabilites que Naive Bayes associe a chaque "racine" et ainsi ameliorer ses predictions. Dans les faits, on observe que l'utilisation de cette methode permet une amelioration des predictions bien que celle-ci soit relativement limitee. Cette technique permet d'extraire l'information liee au fond du commentaire, leur contexte et leur forme ne sont cependant pas ideals pour ce genre de methode (language familier, nombreuses fautes de langue, fautes de frappe, codes, ...).  
L'accuracy est proche de 85% et semble plus stable.

### Question 4

In [20]:
# /!\ L'utilisation de nltk a nécéssité une installation
# de nltk 3.2.1 via 'conda update'
# Une erreur se produit pour la fonction pos_tag()
# avec la version 3.2.0 
# (obtenue avec la commande "nltk.download('all')")
from nltk import pos_tag, word_tokenize

In [21]:
# liste des codes pour les noms, verbes et adjectifs
NVAD = ['NN', 'JJ', 'JJR', 'JJS', 'NN',
        'NNP', 'NNS', 'NNPS', 'RB', 'RBR',
        'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
pos_texts = [" ".join([word[0] for word
                       in pos_tag(word_tokenize(sentence))
                       if (word[1] in NVAD)]) for sentence in texts]

display_CVResults(LR_model, pos_texts, 5)

Scores de la validation croisee :
 0: 86.50%
1: 86.00%
2: 84.25%
3: 84.75%
4: 84.00% 
 IC de l'accuracy : 85.10% +/- 0.9823%


### Analyse et test sur les methodes de vectorisation

In [24]:
wd_CntVect = CountVectorizer(stop_words={'english'}, analyzer='char')

cl_vocabulary, cl_X = count_words(texts, True)
wd_CntVect.fit(texts)

diff = list(set(cl_vocabulary.keys()) - set(wd_CntVect.get_feature_names()))

print("\nNombre de features avec CountVectorizer() :",
      len(wd_CntVect.get_feature_names()),
      "\nNombre de features avec count_words :",
      len(set(cl_vocabulary.keys())),
      "\nNombre de mots différents approximatifs :",
      len(diff))
      


Nombre de features avec CountVectorizer() : 73 
Nombre de features avec count_words : 50375 
Nombre de mots différents approximatifs : 50336


In [25]:
char_CntVect = CountVectorizer(stop_words={'english'}, analyzer='char')
res = char_CntVect.fit_transform(texts)
print(char_CntVect.get_feature_names())

['\x05', '\x12', '\x13', '\x14', '\x16', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']


In [26]:
print(res[:1000,:].sum(axis=0), res[1000:,:].sum(axis=0))

[[     2     33      7      7     12 705709   1056   9120     26    109
      36    192  15345   5650   5742    670     63  35269   9670  32162
     470   1529   1560    630    397    336    376    290    330    484
    1268   1540    876    240     18   2201     12     44      3     44
       3    573    220 223499  46694  83606  95302 338479  62165  60172
  150770 209470   7172  25390 126738  76512 190446 207447  52282   2562
  162031 193348 256611  76181  32849  51563   5032  55447   2918      0
       0      0      1]] [[     4     11      0      0      0 787106    657   8492     29    112
      23    117  15280   6014   6039    384     95  42448  10070  33714
     515   1399   1728    678    408    313    422    296    485    451
    1486   1502    974    319     25   1570      5     46      8     46
       2    495    277 257030  50316  96508 106545 384360  73883  67057
  169969 241671   7611  26962 143817  87534 216771 229597  57059   3180
  188651 222104 287907  83403  36258  5